The first step is to create an AI that can play the mineral collection game.

In [12]:
import numpy as np
from pysc2.agents import base_agent
from pysc2.lib import actions
from pysc2.lib import features
from pysc2.env import sc2_env, run_loop, available_actions_printer
from pysc2 import maps
from absl import flags

Setup useful constants for actions/things:

In [8]:
_AI_RELATIVE = features.SCREEN_FEATURES.player_relative.index
_AI_SELF = 1
_AI_NEUTRAL = 3
_NO_OP = actions.FUNCTIONS.no_op.id
_MOVE_SCREEN = actions.FUNCTIONS.Attack_screen.id
_SELECT_ARMY = actions.FUNCTIONS.select_army.id
_AI_RELATIVE, _AI_SELF, _AI_NEUTRAL, _NO_OP, _MOVE_SCREEN, _SELECT_ARMY

(5, 1, 3, 0, 12, 7)

What are these numbers? They are indices that get particular data or functions. [You can read about it here](https://github.com/deepmind/pysc2/blob/master/docs/environment.md#screen).

The index below is for passing into `obs.observation['screen'][_THE_INDEX]`

`_AI_RELATIVE` - is an index for obs.observation['screen'] (what our AI can see about the wrold). It gets one of the possible obsevation spaces from the screen ([see this for the rest](https://www.youtube.com/watch?v=-fKUyT14G-8)). The grid we get is the 1st screen in the second row (index 5); again this is the world our AI will see. It's an 84x84 grid that has 0, 1, 2, 3, 4s, 0 - background, 1 - the AI's units, 2 - allied units, 3 - neutral units, 4 enemy (hostile) units. We shouldn't have any 2s, 4s in our game but we will have 0,1,3

`_AI_SELF` - just tells you what the value of its own objects are in the grid world our AI will see, as per the docs it should be 1

`_AI_NEUTRAL` - just tells you what the value of neutral objects are in the grid world our AI will see, as per the docs it should be 3

The indices below are for passing into `actions.FunctionCall(_THE_INDEX, [FLAGS])`

`_MOVE_SCREEN` - is an index tells your AI to move whatever it has selected somewhere on the current screen 

`_SELECT_ARMY` - tells your AI to select its units

`_NO_OP` - is an action your AI can perform, the action to do nothing, its like conciously choosing to do nothing if that's the best thing to do.

To see all possible actions run this in your terminal: 

`python -m pysc2.bin.valid_actions`

If this is still confusing don't worry it will make sense in a second.

In [9]:
_SELECT_ALL = [0]
_NOT_QUEUED = [0]

These are flags for passing into FLAGS: `actions.FunctionCall(_THE_INDEX, [FLAGS])`.

The `_SELECT_ALL` flag set to 0 is used to say whether to add the unit to the current selection or replace it. So if it's set to `0` then we should just replace whatever is current selected by reselecting the army, if it's set to `1` we should select the army in addition to whatever else we have selected. This is for when you may want to select non-combat units + army units.

THE `_NOT_QUEUED` flag set to `0` will be used say 'don't queue this action, just do it right away.'Queueing allows the creation of bots with more complex behavior by adding actions that can be delayed or put off for later (until there is nothing happening or synchronize actions on a timer: build a marine at 5, 20, 100, etc).

Alright enough blab, let's build the bot.

In [10]:
def get_marine_location(ai_relative_view):
    '''get the indices where the world is equal to 1'''
    return (ai_relative_view == _AI_SELF).nonzero()
    
def get_rand_location(ai_location):
    '''gets a random location at least n away from current x,y point.'''
    ai_x, ai_y = ai_location[0].mean(), ai_location[1].mean()
    move_at_leastx = 0
    move_at_leasty = 0
    # get dircetion to move
    if ai_x < 32:
        move_at_leastx = 5
    else:
        move_at_leastx = -5
        
    if ai_y < 32:
        move_at_leasty = 5
    else:
        move_at_leasty = -5
    return [np.random.randint(ai_x+move_at_leastx, 64), np.random.randint(ai_y+move_at_leasty, 64)]
    
class Agent(base_agent.BaseAgent):
    """An agent for doing a simple movement form one point to another."""
    def step(self, obs):
        '''step function gets called automatically by pysc2 environment'''
        # call the parent class to have pysc2 setup rewards/etc for us
        super(Agent, self).step(obs)
        # if we can move our army (we have something selected)
        if _MOVE_SCREEN in obs.observation['available_actions']:
            # get what the ai can see about the world
            ai_view = obs.observation['screen'][_AI_RELATIVE]
            # get the location of our marine in this world
            marine_x, marine_y = get_marine_location(ai_view)
            # it our marine is not on the screen do nothing.
            # this happens if we scroll away and look at a different
            # part of the world
            if not marine_x.any():
                return actions.FunctionCall(_NO_OP, [])
            target = get_rand_location([marine_x, marine_y])
            return actions.FunctionCall(_MOVE_SCREEN, [_NOT_QUEUED, target])
        # if we can't move, we havent selected our army, so selecto ur army
        else:
            return actions.FunctionCall(_SELECT_ARMY, [_SELECT_ALL])

In [11]:
FLAGS = flags.FLAGS
FLAGS(['run_sc2'])

steps = 5000
save_replay = True

# create a map
beacon_map = maps.get('MoveToBeacon')

# create an envirnoment
with sc2_env.SC2Env(agent_race=None, bot_race=None, difficulty=None, map_name=beacon_map) as env:
    agent=Agent()
    run_loop.run_loop([agent], env, steps)
    if save_replay:
        env.save_replay(Agent.__name__)

W1231 15:08:09.938910 140160288827136 sc_process.py:183] Killing the process.


Took 15.398 seconds for 5000 steps: 324.720 fps


Other sources:
https://github.com/chris-chris/pysc2-examples

https://github.com/skjb/pysc2-tutorial

https://chatbotslife.com/building-a-basic-pysc2-agent-b109cde1477c

https://gamescapad.es/building-bots-in-starcraft-2-for-psychologists/#installation